In [1]:
import time
import os
import random
import numpy as np                                                
import matplotlib.pyplot as plt                                   
import autograd, autograd.core, autograd.extend, autograd.tracer  
import autograd.numpy as anp      
import scipy, scipy.ndimage, scipy.sparse, scipy.sparse.linalg    
                                                     
import gym
from gymnasium import spaces

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter

from skimage.transform import resize
import torch
import torch.nn as nn
import torch.nn.functional as F

The below code is my RL extension to https://www.researchgate.net/publication/360698153_A_Tutorial_on_Structural_Optimization 

In [2]:
x0,y0=8,8

In [3]:
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d
    
def get_args(normals, forces, density=1e-4):  # Manage the problem setup parameters
    width = normals.shape[0] - 1
    height = normals.shape[1] - 1
    fixdofs = np.flatnonzero(normals.ravel())
    alldofs = np.arange(2 * (width + 1) * (height + 1))
    freedofs = np.sort(list(set(alldofs) - set(fixdofs)))
    params = {
      # material properties
      'young': 1, 'young_min': 1e-9, 'poisson': 0.3, 'g': 0,
      # constraints
      'density': density, 'xmin': 0.001, 'xmax': 1.0,
      # input parameters
      'nelx': width, 'nely': height, 'mask': 1, 'penal': 3.0, 'filter_width': 1,
      'freedofs': freedofs, 'fixdofs': fixdofs, 'forces': forces.ravel(),
      # optimization parameters
      'opt_steps': 80, 'print_every': 10}
    return ObjectView(params)

def mbb_beam(width = y0, height = x0, density=1e-4, y=1, x=0, rd=-1):  # textbook beam example
    normals = np.zeros((width + 1, height + 1, 2))
    normals[0, 0, x] = 1
    normals[0, 0, y] = 1
    normals[0, -1, x] = 1
    normals[0, -1, y] = 1
    forces = np.zeros((width + 1, height + 1, 2))
    forces[-1, rd, y] = -1
    return normals, forces, density

In [4]:
def young_modulus(x, e_0, e_min, p=3):
    return e_min + x ** p * (e_0 - e_min)

def physical_density(x, args, volume_contraint=False, use_filter=True):
    x = args.mask * x.reshape(args.nely, args.nelx)  # reshape from 1D to 2D
    return gaussian_filter(x, args.filter_width) if use_filter else x  # maybe filter

def mean_density(x, args, volume_contraint=False, use_filter=True):
    return anp.mean(physical_density(x, args, volume_contraint, use_filter)) / anp.mean(args.mask)

In [5]:
def objective(x, args, volume_contraint=False, use_filter=True):
    kwargs = dict(penal=args.penal, e_min=args.young_min, e_0=args.young)
    x_phys = physical_density(x, args, volume_contraint=volume_contraint, use_filter=use_filter)
    ke     = get_stiffness_matrix(args.young, args.poisson)  # stiffness matrix
    u      = displace(x_phys, ke, args.forces, args.freedofs, args.fixdofs, **kwargs)
    c      = compliance(x_phys, u, ke, **kwargs)
    return c

In [6]:
@autograd.extend.primitive
def gaussian_filter(x, width): # 2D gaussian blur/filter
    return scipy.ndimage.gaussian_filter(x, width, mode='reflect')

def _gaussian_filter_vjp(ans, x, width): # gives the gradient of orig. function w.r.t. x
    del ans, x  # unused
    return lambda g: gaussian_filter(g, width)
autograd.extend.defvjp(gaussian_filter, _gaussian_filter_vjp)

In [7]:
def compliance(x_phys, u, ke, *, penal=3, e_min=1e-9, e_0=1):
    nely, nelx = x_phys.shape
    ely, elx = anp.meshgrid(range(nely), range(nelx))  # x, y coords for the index map

    n1 = (nely+1)*(elx+0) + (ely+0)  # nodes
    n2 = (nely+1)*(elx+1) + (ely+0)
    n3 = (nely+1)*(elx+1) + (ely+1)
    n4 = (nely+1)*(elx+0) + (ely+1)
    all_ixs = anp.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
    u_selected = u[all_ixs]  # select from u matrix

    ke_u = anp.einsum('ij,jkl->ikl', ke, u_selected)  # compute x^penal * U.T @ ke @ U
    ce = anp.einsum('ijk,ijk->jk', u_selected, ke_u)
    C = young_modulus(x_phys, e_0, e_min, p=penal) * ce.T
    return anp.sum(C)

def get_stiffness_matrix(e, nu):  # e=young's modulus, nu=poisson coefficient
    k = anp.array([1/2-nu/6, 1/8+nu/8, -1/4-nu/12, -1/8+3*nu/8,
                -1/4+nu/12, -1/8-nu/8, nu/6, 1/8-3*nu/8])
    return e/(1-nu**2)*anp.array([[k[0], k[1], k[2], k[3], k[4], k[5], k[6], k[7]],
                               [k[1], k[0], k[7], k[6], k[5], k[4], k[3], k[2]],
                               [k[2], k[7], k[0], k[5], k[6], k[3], k[4], k[1]],
                               [k[3], k[6], k[5], k[0], k[7], k[2], k[1], k[4]],
                               [k[4], k[5], k[6], k[7], k[0], k[1], k[2], k[3]],
                               [k[5], k[4], k[3], k[2], k[1], k[0], k[7], k[6]],
                               [k[6], k[3], k[4], k[1], k[2], k[7], k[0], k[5]],
                               [k[7], k[2], k[1], k[4], k[3], k[6], k[5], k[0]]])

In [8]:
def get_k(stiffness, ke):
    # Constructs sparse stiffness matrix k (used in the displace fn)
    # First, get position of the nodes of each element in the stiffness matrix
    nely, nelx = stiffness.shape
    ely, elx = anp.meshgrid(range(nely), range(nelx))  # x, y coords
    ely, elx = ely.reshape(-1, 1), elx.reshape(-1, 1)

    n1 = (nely+1)*(elx+0) + (ely+0)
    n2 = (nely+1)*(elx+1) + (ely+0)
    n3 = (nely+1)*(elx+1) + (ely+1)
    n4 = (nely+1)*(elx+0) + (ely+1)
    edof = anp.array([2*n1, 2*n1+1, 2*n2, 2*n2+1, 2*n3, 2*n3+1, 2*n4, 2*n4+1])
    edof = edof.T[0]
    x_list = anp.repeat(edof, 8)  # flat list pointer of each node in an element
    y_list = anp.tile(edof, 8).flatten()  # flat list pointer of each node in elem

    # make the global stiffness matrix K
    kd = stiffness.T.reshape(nelx*nely, 1, 1)
    value_list = (kd * anp.tile(ke, kd.shape)).flatten()
    return value_list, y_list, x_list

def displace(x_phys, ke, forces, freedofs, fixdofs, *, penal=3, e_min=1e-9, e_0=1):
    # Displaces the load x using finite element techniques (solve_coo=most of runtime)
    stiffness = young_modulus(x_phys, e_0, e_min, p=penal)
    k_entries, k_ylist, k_xlist = get_k(stiffness, ke)

    index_map, keep, indices = _get_dof_indices(freedofs, fixdofs, k_ylist, k_xlist)

    u_nonzero = solve_coo(k_entries[keep], indices, forces[freedofs], sym_pos=True)
    u_values = anp.concatenate([u_nonzero, anp.zeros(len(fixdofs))])
    return u_values[index_map]

In [9]:
def _get_dof_indices(freedofs, fixdofs, k_xlist, k_ylist):
    index_map = inverse_permutation(anp.concatenate([freedofs, fixdofs]))
    keep = anp.isin(k_xlist, freedofs) & anp.isin(k_ylist, freedofs)
    # Now we index an indexing array that is being indexed by the indices of k
    i = index_map[k_ylist][keep]
    j = index_map[k_xlist][keep]
    return index_map, keep, anp.stack([i, j])

def inverse_permutation(indices):  # reverses an index operation
    inverse_perm = np.zeros(len(indices), dtype=anp.int64)
    inverse_perm[indices] = np.arange(len(indices), dtype=anp.int64)
    return inverse_perm

In [10]:
def _get_solver(a_entries, a_indices, size, sym_pos):
    # a is (usu.) symmetric positive; could solve 2x faster w/sksparse.cholmod.cholesky(a).solve_A
    a = scipy.sparse.coo_matrix((a_entries, a_indices), shape=(size,)*2).tocsc()
    return scipy.sparse.linalg.splu(a).solve

@autograd.primitive
def solve_coo(a_entries, a_indices, b, sym_pos=False):
    solver = _get_solver(a_entries, a_indices, b.size, sym_pos)
    return solver(b)

def grad_solve_coo_entries(ans, a_entries, a_indices, b, sym_pos=False):
    def jvp(grad_ans):
        lambda_ = solve_coo(a_entries, a_indices if sym_pos else a_indices[::-1],
                            grad_ans, sym_pos)
        i, j = a_indices
        return -lambda_[i] * ans[j]
    return jvp

autograd.extend.defvjp(solve_coo, grad_solve_coo_entries,
                       lambda: print('err: gradient undefined'),
                       lambda: print('err: gradient not implemented'))

In [11]:
class Model:
    def __init__(self, shape):
        self.flag_ = True
#         self.flag_ = False
        self.n, self.m = shape
        self.actions_dic={} 
    
        k=0
        for i in range(self.n):
            for j in range(self.m):
                self.actions_dic[k]=(i,j)
                k+=1
        
    def action_space_(self, x, y, X):
#         x,y=self.actions_dic[action]
        X[x][y]=1
        
    def draw(self,X):  
        plt.figure(dpi=50) 
        print('\nFinal Cantilever beam design:')
        plt.imshow(X) 
        plt.show()

In [12]:
def fast_stopt(args, x):

    reshape = lambda x: x.reshape(args.nely, args.nelx)
    objective_fn = lambda x: objective(reshape(x), args)
#     constraint = lambda params: mean_density(reshape(params), args) - args.density
    constraint = lambda params: mean_density(reshape(params), args) 
    value = objective_fn(x)
    const = constraint(x)
    return value, const

In [45]:
class CantileverEnv(gym.Env):
    
    metadata = {"render.modes": ["human"]}

    def __init__(self, normalized_image: bool = False):
        super().__init__()
        
        
        self.rd=0
        self.args = get_args(*mbb_beam(rd=self.rd))
        
#         DIM=self.args.nelx*self.args.nely
#         N_DISCRETE_ACTIONS=self.args.nelx*self.args.nely
        N_DISCRETE_ACTIONS=2*2
        
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        self.observation_space = spaces.Box(low=0,
                                            high=255,
                                            shape=(self.args.nely, self.args.nelx, 1),
                                            dtype=np.float64)
        
 
        self.x = anp.ones((self.args.nely, self.args.nelx))*self.args.density
    
        self.M=Model((2,2))
        
        self.reward=0
        self.step_=0
        self.needs_reset = True
        self.level=1
        self.save_x, self.save_y = 0, 0
        
    def step(self, action):
        
        self.args = get_args(*mbb_beam(rd=self.rd))
        
        if self.level==1:
            self.save_x, self.save_y = self.M.actions_dic[action]
            self.level-=1
            self.reward+=0
            const=0
        else:
            x_c, y_c = self.M.actions_dic[action]
            x_c, y_c=x_c+2*self.save_x, y_c+2*self.save_y 
            self.M.action_space_(x_c, y_c, self.x)
            self.level=1
            
            tmp, const = fast_stopt(self.args, self.x)
        
            self.step_+=1
        
            self.reward+=-np.sqrt(tmp)
#         print(self.reward)    
       
        done=False
                  
        if self.step_>self.M.n*self.M.m:
            done=True
            
        if const>0.65:
#             self.reward-=1
            done=True
            
        if self.needs_reset:
            raise RuntimeError("Tried to step environment that needs reset")
            
        if done:
            self.needs_reset = True
                         
                
#         resized_x = resize(self.x, ( self.args.nely, self.args.nelx), anti_aliasing=True)  
#         resized_x = (resized_x * 255).astype(np.uint8).reshape(x0,y0,1)

        return self.x.reshape(x0,y0,1), self.reward, done, False, {}

    def reset(self, seed=0):
        
        if not self.M.flag_:
            self.rd=random.choice([0,2,-2])
        else:
            self.rd=-1
           
        self.x = anp.ones((self.args.nely, self.args.nelx))*self.args.density 

        self.reward=0
        self.needs_reset = False
        self.step_=0
        self.level=1
        
#         resized_x = resize(self.x, (self.args.nely, self.args.nelx), anti_aliasing=True)
#         resized_x = (resized_x * 255).astype(np.uint8).reshape(x0,y0,1)

        return self.x.reshape(x0,y0,1), {}

    def render(self, mode="human"):
        self.M.draw(self.x)    

    def close(self):
        pass

In [46]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

In [47]:
# ts=5e6
ts=3e6

In [48]:
# Create log dir
log_dir = "/tmp/gym8_1/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = CantileverEnv()
# Logs will be saved in log_dir/monitor.csv
env = Monitor(env, log_dir)
check_env(env)

In [49]:
# callback = SaveOnBestTrainingRewardCallback(check_freq=5000, log_dir=log_dir)
callback = SaveOnBestTrainingRewardCallback(check_freq = 10000, log_dir = log_dir)

In [ ]:
start = time.time()
model = PPO("MlpPolicy", env).learn(total_timesteps=ts, callback=callback)
# model = PPO.load(log_dir + "best_model.zip", env=env).learn(total_timesteps=ts, callback=callback)
end = time.time()   

Num timesteps: 10000
Best mean reward: -inf - Last mean reward per episode: -2405423.00
Saving new best model to /tmp/gym8_1/best_model.zip
Num timesteps: 20000
Best mean reward: -2405423.00 - Last mean reward per episode: -2410618.70
Num timesteps: 30000
Best mean reward: -2405423.00 - Last mean reward per episode: -2400306.80
Saving new best model to /tmp/gym8_1/best_model.zip
Num timesteps: 40000
Best mean reward: -2400306.80 - Last mean reward per episode: -2401706.78
Num timesteps: 50000
Best mean reward: -2400306.80 - Last mean reward per episode: -2406129.30
Num timesteps: 60000
Best mean reward: -2400306.80 - Last mean reward per episode: -2414166.22
Num timesteps: 70000
Best mean reward: -2400306.80 - Last mean reward per episode: -2398813.45
Saving new best model to /tmp/gym8_1/best_model.zip
Num timesteps: 80000
Best mean reward: -2398813.45 - Last mean reward per episode: -2401725.12
Num timesteps: 90000
Best mean reward: -2398813.45 - Last mean reward per episode: -2404533

Num timesteps: 840000
Best mean reward: -2392985.48 - Last mean reward per episode: -2398211.53
Num timesteps: 850000
Best mean reward: -2392985.48 - Last mean reward per episode: -2400511.65
Num timesteps: 860000
Best mean reward: -2392985.48 - Last mean reward per episode: -2407621.26
Num timesteps: 870000
Best mean reward: -2392985.48 - Last mean reward per episode: -2411118.31
Num timesteps: 880000
Best mean reward: -2392985.48 - Last mean reward per episode: -2402331.00
Num timesteps: 890000
Best mean reward: -2392985.48 - Last mean reward per episode: -2400744.03
Num timesteps: 900000
Best mean reward: -2392985.48 - Last mean reward per episode: -2402276.79
Num timesteps: 910000
Best mean reward: -2392985.48 - Last mean reward per episode: -2401743.97
Num timesteps: 920000
Best mean reward: -2392985.48 - Last mean reward per episode: -2401157.53
Num timesteps: 930000
Best mean reward: -2392985.48 - Last mean reward per episode: -2405272.74
Num timesteps: 940000
Best mean reward: 

Num timesteps: 1690000
Best mean reward: -2388037.79 - Last mean reward per episode: -2407972.16
Num timesteps: 1700000
Best mean reward: -2388037.79 - Last mean reward per episode: -2404151.69
Num timesteps: 1710000
Best mean reward: -2388037.79 - Last mean reward per episode: -2405349.49
Num timesteps: 1720000
Best mean reward: -2388037.79 - Last mean reward per episode: -2404021.52
Num timesteps: 1730000
Best mean reward: -2388037.79 - Last mean reward per episode: -2389622.06
Num timesteps: 1740000
Best mean reward: -2388037.79 - Last mean reward per episode: -2398644.69
Num timesteps: 1750000
Best mean reward: -2388037.79 - Last mean reward per episode: -2404293.20
Num timesteps: 1760000
Best mean reward: -2388037.79 - Last mean reward per episode: -2398993.49
Num timesteps: 1770000
Best mean reward: -2388037.79 - Last mean reward per episode: -2401149.13
Num timesteps: 1780000
Best mean reward: -2388037.79 - Last mean reward per episode: -2406053.98
Num timesteps: 1790000
Best me

Num timesteps: 2540000
Best mean reward: -2388037.79 - Last mean reward per episode: -2400435.27
Num timesteps: 2550000
Best mean reward: -2388037.79 - Last mean reward per episode: -2404492.53
Num timesteps: 2560000
Best mean reward: -2388037.79 - Last mean reward per episode: -2404414.27
Num timesteps: 2570000
Best mean reward: -2388037.79 - Last mean reward per episode: -2401286.04
Num timesteps: 2580000
Best mean reward: -2388037.79 - Last mean reward per episode: -2410916.66
Num timesteps: 2590000
Best mean reward: -2388037.79 - Last mean reward per episode: -2402300.92
Num timesteps: 2600000
Best mean reward: -2388037.79 - Last mean reward per episode: -2396816.15
Num timesteps: 2610000
Best mean reward: -2388037.79 - Last mean reward per episode: -2407404.73
Num timesteps: 2620000
Best mean reward: -2388037.79 - Last mean reward per episode: -2398685.30
Num timesteps: 2630000
Best mean reward: -2388037.79 - Last mean reward per episode: -2401816.12
Num timesteps: 2640000
Best me

In [ ]:
print('Total time taken: {} min'.format((end - start)/60))

### Inference

In [ ]:
# env.M.flag_=True
env.M.flag_=False
obs=env.reset()
obs=obs[0]

In [ ]:
i=0
while i<2000:
    action, _states = model.predict(obs)
    obs, rewards, dones, _ , info = env.step(int(action))
    if dones: 
        break
    i+=1

In [ ]:
print(i)

In [ ]:
fast_stopt(env.args, env.x)

In [ ]:
env.render()

In [ ]:
results_plotter.plot_results([log_dir], ts, results_plotter.X_TIMESTEPS, "CantileverEnv")

In [ ]:
obs=env.reset()
obs=obs[0]

In [ ]:
# Load the agent
model_best = PPO.load(log_dir + "best_model.zip", env=env)

In [ ]:
i=0
while i<2000:
    action, _states = model_best.predict(obs)
    
    obs, rewards, dones, _ , info = env.step(int(action))
    if dones:
        break
    i+=1

In [ ]:
i

In [ ]:
env.render()

In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/save_format.html

In [ ]:
fast_stopt(env.args, env.x)